In [62]:
# Check GPU

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [63]:
# Load Dataset
# Note: this is a workaround to bypass known issue:
# Failed to download (trying next): <urlopen error [Errno 110] Connection timed out>
import torchvision.datasets as datasets
import socket

socket.setdefaulttimeout(5)

data_root = './data'

train_set0 = datasets.MNIST(
    root = data_root,
    train = True,
    download = True
)

In [64]:
# Preprocess Dataset
import torchvision.transforms as transforms

transform = transforms.Compose([
    # this transforms PIL.Image data into FloatTensor
    # changes each pixel value into 0.0 ... 1.0
    transforms.ToTensor(),
    # this normalizes each tensors into -1 ... 1
    # normalized_output = (input - avg) / std deviation
    # input 0.0 ... 1.0 so output is -1 ... 1
    transforms.Normalize(0.5, 0.5),
    # flatten because l1 is FC Layer
    transforms.Lambda(lambda x: x.view(-1))
])

In [65]:
# Load and preprocess train dataset from MNIST
train_set = datasets.MNIST(
    root = data_root, train = True,
    download = True, transform = transform
    )

In [66]:
# Load and preprocess test dataset from MNIST
test_set = datasets.MNIST(
    root = data_root, train = False,
    download = True, transform = transform
    )

In [67]:
# Chunk train, test dataset into mini batch size.
# Mini-batch helps GD avoid local minima because it adds randomness
# when updating the gradient. Also, shuffle helps the model avoid 
# learning noise from the order of dataset.

from torch.utils.data import DataLoader

# Mini-batch size
batch_size = 500

train_loader = DataLoader(
    train_set, batch_size = batch_size,
    # shuffle only for train to avoid learning
    # noise from the order of dataset
    shuffle = True
    )

test_loader = DataLoader(
    test_set, batch_size = batch_size,
    # does not require shuffle, because we don't update
    # the model parameter while validating
    shuffle = False
    )

In [68]:
# set the input, output, hidden sizes
import numpy as np
image, _ = train_set[0]
n_input = image.shape[0]
n_output = len(train_set0.classes)
n_hidden = 128

In [69]:
# define the NN model (FC Layer with ReLU)

import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        return x3

In [70]:
# initialize NN

net = Net(n_input, n_output, n_hidden).to(device)

In [71]:
print(net)

Net(
  (l1): Linear(in_features=784, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU(inplace=True)
)


In [72]:
for parameter in net.named_parameters():
    print(parameter)

('l1.weight', Parameter containing:
tensor([[-0.0111,  0.0211,  0.0067,  ...,  0.0282,  0.0356, -0.0220],
        [-0.0291, -0.0093, -0.0304,  ...,  0.0054, -0.0076,  0.0129],
        [ 0.0084,  0.0202, -0.0305,  ...,  0.0146,  0.0057, -0.0161],
        ...,
        [-0.0174,  0.0172, -0.0101,  ...,  0.0084,  0.0011,  0.0103],
        [-0.0299, -0.0187, -0.0180,  ...,  0.0122, -0.0326, -0.0079],
        [ 0.0175, -0.0126,  0.0212,  ..., -0.0032, -0.0081,  0.0093]],
       device='cuda:0', requires_grad=True))
('l1.bias', Parameter containing:
tensor([ 0.0092, -0.0268, -0.0168,  0.0035,  0.0045,  0.0210, -0.0231, -0.0174,
         0.0137,  0.0195,  0.0066, -0.0076,  0.0289, -0.0040, -0.0153,  0.0230,
         0.0335, -0.0096, -0.0021, -0.0020,  0.0082,  0.0120, -0.0273, -0.0338,
        -0.0220,  0.0289, -0.0003,  0.0140, -0.0059, -0.0146, -0.0156,  0.0345,
        -0.0028, -0.0109,  0.0269,  0.0259, -0.0119,  0.0266, -0.0074,  0.0159,
         0.0184,  0.0218, -0.0014,  0.0277, -0.0124

In [73]:
# set parameters for training, validating

from torch import optim

history = np.zeros((0,5))
num_epochs = 50
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = optim.SGD(net.parameters(), lr=lr)

In [74]:
# train and validate
import numpy as np
from tqdm.notebook import tqdm

# 500 x (120 + 20) = 70,000
for epoch in range(num_epochs):
    train_correct, train_loss = 0,0
    val_correct, val_loss = 0,0
    n_train, n_test = 0,0

    # set the model to training mode
    net.train()
    # main difference between train and validation is whether
    # we are going to update the gradient or not
    # 60,000 / batch_size=500 = 120 times
    for inputs, labels in tqdm(train_loader):
        n_train += len(labels) # 0 ~ 120

        # load to CUDA GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # initialize gradient to zero for every step
        optimizer.zero_grad()

        # forward function defined in NN is activated.
        # implement l1 -> relu -> l2 to input
        outputs = net(inputs)
        # SGD on outputs and labels
        loss = criterion(outputs, labels)
        # backward propagation
        loss.backward()
        # update gradient efficiently
        optimizer.step()
        # only get the indice of the max value
        predicted = torch.max(outputs, 1)[1]
        
        train_loss += loss.item()
        train_correct += (predicted == labels).sum().item()

    # set the model to evaluation mode
    net.eval()
    # 10,000 / batch_size=500 = 20 times
    # torch.no_grad disables gradient calculation for validation
    with torch.no_grad():
        for inputs_test, labels_test in tqdm(test_loader):
            n_test += len(labels)

            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            optimizer.zero_grad()

            outputs_test = net(inputs_test)
            loss_test = criterion(outputs_test, labels_test)

            predicted_test = torch.max(outputs_test, 1)[1]

            val_loss += loss_test.item()
            val_correct += (predicted_test == labels_test).sum().item()
    
    train_acc = train_correct / n_train
    val_acc = val_correct / n_test

    print(f"Epoch [{epoch+1}/{num_epochs}], train_loss: {train_loss: .5f}, \
            val_loss: {val_loss: .5f}, train_acc: {train_acc: .5f}, \
            val_acc: {val_acc: .5f}")
    
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc])
    history = np.vstack((history, item))

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/50], train_loss:  218.93081,             val_loss:  26.22046, train_acc:  0.58257,             val_acc:  0.77390


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [49/50], train_loss:  28.86355,             val_loss:  4.77119, train_acc:  0.93213,             val_acc:  0.93230


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [50/50], train_loss:  28.60604,             val_loss:  4.75254, train_acc:  0.93257,             val_acc:  0.93170
